In [1]:
import os, numpy as np, pandas as pd

In [2]:
os.chdir("E:\Machine learning course\Raw data")

In [3]:
nortel = pd.read_csv("nortel.csv")

In [4]:
nortel.head()

,Acc_no,REG_No,Reg_date,Comp_date,Month_year,RP,RS,Unit,IR,Performance,OverallPer,t_respond,Understand,t_resolve,s_equipment,s_Del,speak,adv_status
0,CST-H514-H3H,NaN,5/14/2004,5/25/2005,2005-05,4 - L,4-C_problem,GPK,Solved,None,60,60,60,60,NaN,NaN,60.0,60.0
1,CST-H7H6-HH28,CZK3B39B,6/7/2004,8/6/2005,2005-06,3 - M,3-Minor,GPK,Solved,None,80,90,90,90,NaN,80.0,90.0,90.0
2,CST-H9H1-HH86,NaN,1/9/2004,3/5/2005,2005-05,3 - M,4-C_problem,RPLCDS,Solved,Red,40,100,100,50,NaN,NaN,90.0,100.0
3,CST-1H29-H233,NaN,10/29/2004,5/27/2005,2005-05,2 - H,2-Major,RPLCDS,Solved,None,60,90,90,80,NaN,NaN,80.0,80.0
4,CST-11H2-H312,CZK26998,2/11/2004,9/5/2005,2005-05,3 - M,3-Minor,GPK,Solved,Red,40,80,80,10,NaN,100.0,90.0,90.0


### onesample t test

In [6]:
from scipy.stats import ttest_1samp

In [7]:
tsocre,pvalue= ttest_1samp(nortel["OverallPer"],popmean=75)

In [8]:
tsocre,pvalue

(5.49820213421482, 4.141831983513732e-08)

In [9]:
nortel["OverallPer"].mean()

77.73316471704078

In [10]:
tsocre,pvalue= ttest_1samp(nortel[nortel["Unit"]=="GPK"]["OverallPer"],popmean=70)

In [11]:
tsocre,pvalue

(15.443150756614052, 2.233460456401153e-48)

In [12]:
nortel[nortel["Unit"]=="GPK"]["OverallPer"].mean()

82.12487411883183

## 2 sample t test

In [13]:
Overall_IR_yes= nortel[nortel['IR']=='Solved']['OverallPer']
Overall_IR_No= nortel[nortel['IR']=='No']['OverallPer']

In [14]:
from scipy.stats import stats
stats.ttest_ind(a=Overall_IR_yes,b = Overall_IR_No, equal_var=True) # there is significant diff bw overall per when issue is resolved n not resolved

Ttest_indResult(statistic=17.17769278863142, pvalue=2.674032026820951e-63)

In [15]:
nortel[nortel['IR']=='Solved']['OverallPer'].mean()

79.47768595041322

In [16]:
nortel[nortel['IR']=='No']['OverallPer'].mean()

39.492753623188406

In [17]:
nortel[nortel['IR']=='Solved']['OverallPer'].var()

693.2456163365257

In [18]:
nortel[nortel['IR']=='No']['OverallPer'].var()

1197.5510419972486

In [19]:
# give equal_var= false when there huge difference in variance, if not give it as true, this you get when u cal variance values
stats.ttest_ind(a=Overall_IR_yes,b = Overall_IR_No, equal_var=False)

Ttest_indResult(statistic=13.397653222632687, pvalue=4.175124306318692e-27)

##  2 more than 2 samples means ANOVA

In [20]:
# to check if avg overall per differs bw three groups- GPA,GPK, RPLCDS

In [21]:
!pip install researchpy

In [22]:
import researchpy as rp

In [23]:
rp.summary_cont(nortel['OverallPer'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,OverallPer,3163.0,77.7332,27.9573,0.4971,76.7585,78.7078


In [24]:
# the CI says the performance can be 76.75 and can go upto 78.7 - interval estimate, the overall per will range w/i this
# mean gives point estimate = 77.733

In [25]:
rp.summary_cont(nortel['OverallPer'], conf=0.99) # to get 99% CI

,Variable,N,Mean,SD,SE,99% Conf.,Interval
0,OverallPer,3163.0,77.7332,27.9573,0.4971,76.4519,79.0144


In [26]:
rp.summary_cont(nortel['OverallPer'].groupby(nortel['Unit']))

,N,Mean,SD,SE,95% Conf.,Interval
Unit,,,,,,
GPA,11,70.9091,17.0027,5.1265,59.4865,82.3316
GPK,993,82.1249,24.7409,0.7851,80.5842,83.6656
Not,3,56.6667,51.3160,29.6273,-70.8094,184.1427
RPLCDS,2156,75.7746,29.1154,0.6270,74.5449,77.0043


In [27]:
nortel2 = nortel[nortel['Unit']!='Not'] # to remove not
rp.summary_cont(nortel2['OverallPer'].groupby(nortel2['Unit']),conf=0.99)

,N,Mean,SD,SE,99% Conf.,Interval
Unit,,,,,,
GPA,11,70.9091,17.0027,5.1265,54.6618,87.1564
GPK,993,82.1249,24.7409,0.7851,80.0986,84.1511
RPLCDS,2156,75.7746,29.1154,0.6270,74.1580,77.3912


In [28]:
stats.f_oneway(nortel['OverallPer'][nortel['Unit']=='GPA'],
            nortel['OverallPer'][nortel['Unit']=='GPK'],
                nortel['OverallPer'][nortel['Unit']=='RPLCDS'])

F_onewayResult(statistic=18.093861515720143, pvalue=1.5368512826350375e-08)

##chi square test

In [29]:
nortel['Overall_sat']= nortel['OverallPer'].map(lambda x :1 if x>=80 else 0)

In [30]:
nortel['Overall_sat'].value_counts()

1    2228
0     935
Name: Overall_sat, dtype: int64

In [31]:
contingency = pd.crosstab(nortel['IR'], nortel['Overall_sat'])

In [32]:
contingency

Overall_sat,0,1
IR,,
No,105,33
Solved,830,2195


In [33]:
!pip install
from scipy.stats import chi2_contingency
c,p,dof, expected = chi2_contingency(contingency)

ERROR: You must give at least one requirement to install (see "pip help install")


In [34]:
c,p,dof, expected

(147.68388794622243,
 5.562015470937976e-34,
 1,
 array([[  40.79355043,   97.20644957],
        [ 894.20644957, 2130.79355043]]))

In [35]:
# reject null hypothesis. therefore the overall_sat and Ir are strongly associated